# ETUDE DYNAMYQUE DES METROS DE PARIS

## Test simulation Ed ligne M3b

In [1]:
%matplotlib inline
import os
import numpy as np
from LibMetroIti_NB import *

if os.name == 'nt':
    L=listeMetroCsv(".\\data\\liste stations.csv") #list([ligne,station,num_station])
    Lex=listeMetroCsvStation(".\\data\\liste stations exceptions-fourche2.csv") #list([ligne,station,num_station])
    Lf=listeMetroCsvStation(".\\data\\liste stations fourche1.csv") #list([ligne,station,num_station])
    Linfos=listeMetroCsvInfos(".\\data\\infos_metro.csv") #list([ligne,nb_arrets,temps de trajet, capacité rame])
else:
    L=listeMetroCsv("./data/liste stations.csv") #list([ligne,station,num_station])
    Lex=listeMetroCsvStation("./data/liste stations exceptions-fourche2.csv") #list([ligne,station,num_station])
    Lf=listeMetroCsvStation("./data/liste stations fourche1.csv") #list([ligne,station,num_station])
    Linfos=listeMetroCsvInfos("./data/infos_metro.csv") #list([ligne,nb_arrets,temps de trajet, capacité rame])

dict_Infos_metro=dictMetroCsvInfos(Linfos)

In [6]:
L_M3b=list()
for var in L:
    if var[0]=="M3b":
        L_M3b.append(var)
print(L_M3b)

L_M3=list()
for var in L:
    if var[0]=="M3":
        L_M3.append(var)
print(L_M3)  

[['M3b', 'PORTE DES LILAS', '1'], ['M3b', 'SAINT-FARGEAU', '2'], ['M3b', 'PELLEPORT', '3'], ['M3b', 'GAMBETTA', '4']]
[['M3', 'PONT DE LEVALLOIS-BECON', '1'], ['M3', 'ANATOLE FRANCE', '2'], ['M3', 'LOUISE MICHEL', '3'], ['M3', 'PORTE DE CHAMPERRET', '4'], ['M3', 'PEREIRE-LEVALLOIS', '5'], ['M3', 'WAGRAM', '6'], ['M3', 'MALESHERBES', '7'], ['M3', 'VILLIERS', '8'], ['M3', 'EUROPE', '9'], ['M3', 'SAINT-LAZARE', '10'], ['M3', 'HAVRE-CAUMARTIN', '11'], ['M3', 'OPERA', '12'], ['M3', 'QUATRE SEPTEMBRE', '13'], ['M3', 'BOURSE', '14'], ['M3', 'SENTIER', '15'], ['M3', 'REAUMUR-SEBASTOPOL', '16'], ['M3', 'ARTS ET METIERS', '17'], ['M3', 'TEMPLE', '18'], ['M3', 'REPUBLIQUE', '19'], ['M3', 'PARMENTIER', '20'], ['M3', 'SAINT-MAUR', '21'], ['M3', 'PERE LACHAISE', '22'], ['M3', 'GAMBETTA', '23'], ['M3', 'PORTE DE BAGNOLET', '24'], ['M3', 'GALLIENI (PARC DE BAGNOLET)', '25']]


In [3]:
#paramètres
temps_max= 3*4*60 #3 fois * 4 mins * 60 sec
nb_utilisateurs = 10

In [4]:
def simply_iti(station_depart,ligne_depart,station_arrive,ligne_arrive):
    code = calcul_itineraire(station_depart,ligne_depart,station_arrive,ligne_arrive)
    if type(code)==type(0):
        return [station_depart,ligne_depart,station_arrive,ligne_depart]
    else:
        l=code.split(".")
        if l[4]=="":
            return [station_depart,ligne_depart,station_arrive,ligne_depart]
        elif str(l[1])==station_depart:
            return [station_depart,ligne_depart,str(l[3]),ligne_depart,station_arrive,ligne_arrive]
        else:
            return [station_depart,ligne_depart,str(l[1]),ligne_depart,str(l[3]),str(l[4]),station_arrive,ligne_arrive]
print(simply_iti("JUSSIEU","M10","PORTE D'ORLEAN","M4"))

['JUSSIEU', 'M10', 'CHATELET', 'M10', "PORTE D'ORLEAN", 'M4']


In [31]:
def calcul_temps(iti_s):
    if len(iti_s)==8:
        return calcul_distance(iti_s[0],iti_s[1],iti_s[2],iti_s[3])*int(dict_Infos_metro[iti_s[1]][0][0])+calcul_distance(iti_s[2],iti_s[5],iti_s[4],iti_s[5])*int(dict_Infos_metro[iti_s[5]][0][0])+calcul_distance(iti_s[4],iti_s[7],iti_s[6],iti_s[7])*int(dict_Infos_metro[iti_s[7]][0][0])
    elif len(iti_s)==6:
        return calcul_distance(iti_s[0],iti_s[1],iti_s[2],iti_s[3])*int(dict_Infos_metro[iti_s[1]][0][0])+calcul_distance(iti_s[2],iti_s[5],iti_s[4],iti_s[5])*int(dict_Infos_metro[iti_s[5]][0][0])
    elif len(iti_s)==4:
         return calcul_distance(iti_s[0],iti_s[1],iti_s[2],iti_s[3])*int(dict_Infos_metro[iti_s[1]][0][0])

In [90]:
def distrib_itineraire(nb_utilisateurs):
    base_utilisateurs=[]
    for i in range(1,nb_utilisateurs+1,1):
        x=0
        y=0
        while (x==y):
            x=L_M3b[np.random.choice(len(L_M3b))][1]
            y=L_M3b[np.random.choice(len(L_M3b))][1]
        xL,yL="",""
        for var in L:
            if var[1]==x:
                xL=var[0]
            elif var[1]==y:
                yL=var[0]
            elif xL!="" and yL!="":
                break
        base_utilisateurs.append([i,simply_iti(x,xL,y,yL),0,calcul_temps(simply_iti(x,xL,y,yL))])
        #[num_utilisateur,[depart, chagement_1, changement_2, arrivee],temps_spawn, temps_total_trajet]
    return base_utilisateurs
distrib_itineraire(2)

[[1, ['GAMBETTA', 'M3', 'SAINT-FARGEAU', 'M3'], 0, 6900],
 [2, ['GAMBETTA', 'M3b', 'PELLEPORT', 'M3b'], 0, 360]]

In [43]:
def initialisation_stations(L):
    base_station=["init"] #[[station,[lignes],nb d'utilisateur],...]
    for station in L:
        test=[]
        for i in range(0,len(base_station),1):
            if not (station[1] in base_station[i][0]):
                test.append(1)
            else:
                test.append(0)
        if not(0 in test):
            base_station.append([station[1],[station[0]]+correspondance(station[1],station[0]),0])
    return base_station[1:]

print(initialisation_stations(L_M3b))

[['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]]


In [78]:
def update_utilisateurs_entrant(base_utilisateur,nouveaux_utilisateurs,t):
    compteur=int(base_utilisateur[-1][0])
    update_utilisateurs=[]
    for i in range(1+compteur,nouveaux_utilisateurs+compteur+1,1):
        x=0
        y=0
        while (x==y):
            x=L_M3b[np.random.choice(len(L_M3b))][1]
            y=L_M3b[np.random.choice(len(L_M3b))][1]
        xL,yL="",""
        for var in L:
            if var[1]==x:
                xL=var[0]
            elif var[1]==y:
                yL=var[0]
            elif xL!="" and yL!="":
                break   
        update_utilisateurs.append([i,simply_iti(x,xL,y,yL),t,t+calcul_temps(simply_iti(x,xL,y,yL))])
        #[num_utilisateur,[depart, chagement_1, changement_2, arrivee],temps_spawn, temps_total_trajet]
    return base_utilisateur+update_utilisateurs

In [87]:
def simulation(nb_utilisateur_init,nb_utilisateur_update,temps_max):
    """
    version test du projet RATP
    {nb_utilisateur_init -> int* personne qui rentrent dans le métro à t=0
    nb_utilisateur_update -> int* personne qui rentrent dans le métro toutes les minutes
    temps_max -> int* temps de la simualation}
    """
    base_station=initialisation_stations(L_M3b)
    time=[] #stockage des base_station par minute
    list_vidage=list()
    for t in range(0,temps_max+1,1):
        if t==0:
             base_utilisateur=distrib_itineraire(nb_utilisateur_init)
        for utilisateur in base_utilisateur:
            if t==utilisateur[3]:
                for station in base_station:
                    if station[0]==utilisateur[1][-2]:
                        list_vidage.append(utilisateur)
        
        for var in list_vidage:
            for i in range(0,len(var[1]),1):
                for station in base_station:
                    if var[1][i]==station:
                        station[2]+=1
            base_utilisateur.remove(var)
        list_vidage.clear()
        
        if t%5==0:
            base_utilisateur=update_utilisateurs_entrant(base_utilisateur,nb_utilisateur_update,t)
        
        if t%==0:
            time.append(base_station)
    return time
    
print(simulation(1,1,50000))



[[['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['PELLEPORT', ['M3b'], 0], ['GAMBETTA', ['M3b', 'M3'], 0]], [['PORTE DES LILAS', ['M3b', 'M11'], 0], ['SAINT-FARGEAU', ['M3b'], 0], ['P

In [69]:
2+int(distrib_itineraire(10)[-1][0])

12

In [77]:
test=[["a","b"],["c","d"]]
for var in test:
    var[0]=0
print(test)

[[0, 'b'], [0, 'd']]


In [54]:
from PIL import Image

im=Image.open("C:\\Users\\Edouard\\Documents\\Python Scripts\\Projet ratp\\data\\logo-ratp.png")
im.show()

# TODO

distrib iti et update_uti
régler porblème mauvaise attibution des lignes
relire la simulation

In [3]:
listage_stations_itineraire("JUSSIEU","M10","PORTE D'ORLEAN","M4")

'ERROR : code incomplé'